# Exploratory Analysis 

## **Imports**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_data_validation as tfdv

## **Item Info**

In [2]:
item_info_df = pd.read_csv('item_info.csv', dtype={'item_no': str})
display(item_info_df.head())

,item_no,colour,gender_description,brand,product_group,min_age,max_age
0,206890150141030846,beige,unisex,bloomingville,furniture,NaN,NaN
1,7637494654837559066,pink,girls,petit bateau,clothing sets,0.125,2.0
2,7969520735315050609,beige,unisex,bobo choses,jumpers and knitwear,1.000,11.0
3,-565751122846696741,white,unisex,piupiuchick,tops,1.000,14.0
4,6688930722259797984,green,unisex,filibabba,baby feeding,NaN,NaN


In [3]:
item_info_statistics = tfdv.generate_statistics_from_dataframe(item_info_df )
tfdv.visualize_statistics(item_info_statistics)

## **User Info**

In [4]:
user_info_df = pd.read_csv('user_info.csv', dtype={'user_no': str})
user_info_df.head()

,user_no,country,aov,avg_markdown
0,8587933979694308845,sweden,479.2,-0.200000
1,2740387653650048572,sweden,837.6,-0.200000
2,2084988796719274722,sweden,942.4,-0.560000
3,-4662401680846085311,sweden,438.4,-0.620000
4,-1086148156436227367,sweden,664.8,-0.286667


In [5]:
user_info_statistics = tfdv.generate_statistics_from_dataframe(user_info_df)
tfdv.visualize_statistics(user_info_statistics)

## **User Item Interactions**

In [259]:
user_item_interaction_df = pd.read_csv('user_item_interactions.csv', dtype={'user_no': str, 'item_no': str})
display(user_item_interaction_df.head())

,user_no,item_no,date,eventtype
0,9060639138425951676,-478270421339298398,2021-06-24,purchased
1,9060639138425951676,2658388892627023500,2021-11-27,pageView
2,9060639138425951676,504233002877562247,2021-11-26,pageView
3,9060639138425951676,-3413566329152665076,2021-10-21,pageView
4,9060639138425951676,6700254580945881296,2021-10-10,pageView


In [7]:
interaction_statistics = tfdv.generate_statistics_from_dataframe(user_item_interaction_df)
tfdv.visualize_statistics(interaction_statistics)

In [11]:
print(user_item_interaction_df['date'].min())
print(user_item_interaction_df['date'].max())

2021-01-01
2021-11-30


---
## **Tasks**

 - [x] Get poetry working and add TFDV
 - [x] Profile each dataset
 - [x] User/item interactions details 
     - [x] How many page views are turned into purchases? 
     - [x] How many wishlist are turned into purchases
     - [x] Items per basket (big buyers?)
 - [x] Join together 
     - [x] Popular item types? Popular colors? Popular brands? 
 - [ ] KPIs 
     - [ ] Visitors per month
     - [ ] Conversion ratio
     - [ ] Total items sold (per month)
     - [ ] Sessions per month
     - [ ] Sessions with a purchase per month
     - [ ] Returned users 
 - [ ] Individual User Journey 
     - [ ] Dashboard
     - [ ] Buy same type of clothes? 
     - [ ] Can we see them growing up?  
     - [ ] Same brand, same gender?
 - [ ] Cold start, new users/items
 - [ ] Train/test split and baseline
 - [ ] Determination of features
 - [ ] Write out data for training
 - [ ] Clean up and write text (summarize)
     - [ ] How many unique items/users
     - [ ] Most popular items, most frequent users
     - [ ] Event types
 
<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>

<div class="alert alert-block alert-success">
<b>Up to you:</b> Use green boxes sparingly, and only for some specific 
purpose that the other boxes can't cover. For example, if you have a lot 
of related content to link to, maybe you decide to use green boxes for 
related links from each section of a notebook.
</div>

<div class="alert alert-block alert-danger">
<b>Just don't:</b> In general, avoid the red boxes. These should only be
used for actions that might cause data loss or another major issue.
</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> Use yellow boxes for examples that are not 
inside code cells, or use for mathematical formulas if needed.
    
```python
class Dev:
    def __init__(self):
```
</div>

$Hello_p$

---
## **User-Item Interaction Details**

 - [x] Purchases that weren't previously viewed or added to cart
 - [x] Page views/wishlist that turned into purchases
 - [x] Items per basket 

In [31]:
user_item_interaction_jan = user_item_interaction_df.loc[
    user_item_interaction_df['date'].apply(lambda x: x[:7]) == '2021-01']

In [32]:
user_item_interaction_jan

,user_no,item_no,date,eventtype
54,-4992480809564769711,8348016184428027188,2021-01-26,pageView
82,-9047880956100698698,6121207426465721414,2021-01-29,pageView
133,4197025502585486837,-6172044358363580016,2021-01-21,pageView
257,-1603776270012168559,6925252469515807917,2021-01-28,addToCart
258,-1603776270012168559,6194399194013600425,2021-01-28,addToCart
...,...,...,...,...
1011170,2740387653650048572,2559370552162586247,2021-01-17,addToCart
1011195,7068348551466544329,1472365912138241045,2021-01-19,purchased
1011196,7068348551466544329,981298674982707936,2021-01-19,purchased
1011197,7068348551466544329,5972339127440735267,2021-01-19,purchased


In [33]:
user_item_interaction_jan_groups = user_item_interaction_jan.groupby(['user_no', 'item_no'])

In [113]:
pivoted = pd.pivot_table(user_item_interaction_jan, 
                         index=['user_no', 'item_no'], 
                         columns=['eventtype'], 
                         aggfunc='count', 
                         fill_value=0).reset_index()

pivoted.columns = ['user_no', 'item_no', 'addToCart', 'pageView', 'purchased']

In [122]:
purchased_no_page_view = pivoted.loc[(pivoted['purchased'] == 1) & (pivoted['pageView'] == 0)]

In [129]:
pivoted = pd.pivot_table(user_item_interaction_df, 
                         index=['user_no', 'item_no'], 
                         columns=['eventtype'], 
                         aggfunc='count', 
                         fill_value=0).reset_index()

pivoted.columns = ['user_no', 'item_no', 'addToCart', 'addToWishlist', 'pageView', 'purchased']

In [130]:
pivoted

,user_no,item_no,addToCart,addToWishlist,pageView,purchased
0,-1000173980966705625,2356442837089120035,0,0,1,1
1,-1000173980966705625,6194820387173010563,0,0,1,0
2,-1000173980966705625,6283078728673886163,0,0,1,1
3,-1000190704253701773,383205862898605182,1,0,1,0
4,-1001470904269092144,-4036395326208857801,2,0,2,0
...,...,...,...,...,...,...
784712,9999165590351299,-7779279685545513760,0,0,1,0
784713,9999165590351299,169913927963481109,0,0,1,0
784714,9999165590351299,3042466209036108905,0,0,1,0
784715,99995943117292705,-8708142935431931645,0,0,0,1


In [136]:
purchases = pivoted.loc[pivoted['purchased'] == 1]
purchases

,user_no,item_no,addToCart,addToWishlist,pageView,purchased
0,-1000173980966705625,2356442837089120035,0,0,1,1
2,-1000173980966705625,6283078728673886163,0,0,1,1
17,-1001697075369787517,-1365382699650566863,1,0,0,1
19,-1001697075369787517,-7009163308018422334,1,1,1,1
20,-1001697075369787517,-752701044660672673,1,1,2,1
...,...,...,...,...,...,...
784698,999541357744859146,594778003920049884,0,0,0,1
784701,999541357744859146,7793588584802914643,0,0,0,1
784702,999541357744859146,8693370834128739003,0,0,0,1
784706,99973748111773990,-9172288605754916860,0,0,1,1


In [168]:
purchased = pivoted.loc[pivoted['purchased'] == 1]

purchased_no_other_events = purchased.loc[(purchased['addToCart'] == 0) & 
                                          (purchased['addToWishlist'] == 0) & 
                                          (purchased['pageView'] == 0)]

In [171]:
print(f"{len(purchased_no_other_events) / len(purchased):.2%} of purchases had no other events")

58.22% of purchases had no other events


<div class="alert alert-block alert-warning">
<b>Warning:</b> Many purchases have no other events, implications for how we track conversions, e.g. just because few pageViews turn into purchases isn't necessarily bad if we have a bunch of other purchases from unknown provenance.
</div>

**pageView, addToCard, addToWishlist** turned into purchased

In [164]:
def calc_percent_conversion(df, event):
    event_df = df.loc[df[event] > 0]
    purchases = event_df.loc[event_df['purchased'] > 0]
    return len(purchases) / len(event_df)

def print_conversion(df, event):
    conversion_ratio = calc_percent_conversion(df, event)
    print(f"{conversion_ratio:<10.2%} {event:<15} events resulted in a purchase")

In [166]:
print_conversion(pivoted, 'addToCart')
print_conversion(pivoted, 'addToWishlist')
print_conversion(pivoted, 'pageView')

22.29%     addToCart       events resulted in a purchase
8.37%      addToWishlist   events resulted in a purchase
12.74%     pageView        events resulted in a purchase


---

**Items per basket**

In [181]:
purchases = user_item_interaction_df.loc[user_item_interaction_df['eventtype'] == 'purchased']
session_baskets = purchases.groupby(['user_no', 'date']).agg('count').reset_index()[['user_no', 'date', 'item_no']]

In [183]:
session_baskets.describe()

,item_no
count,85452.000000
mean,2.658393
std,2.190587
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,20.000000


---

## **Join Together**

- [ ] Join together 
     - [ ] Popular item types? Popular colors? Popular brands? 

In [194]:
user_item_interaction_w_metadata = pd.merge(pivoted, 
                                            item_info_df,
                                            how='left', 
                                            on='item_no')

In [197]:
purchased = user_item_interaction_w_metadata.loc[user_item_interaction_w_metadata['purchased'] > 0]
not_purchased = user_item_interaction_w_metadata.loc[user_item_interaction_w_metadata['purchased'] == 0]

In [201]:
purchased_statistics = tfdv.generate_statistics_from_dataframe(purchased)
not_purchased_statistics = tfdv.generate_statistics_from_dataframe(not_purchased)

In [204]:
tfdv.visualize_statistics(purchased_statistics, 
                          not_purchased_statistics, 
                          lhs_name='purchased', 
                          rhs_name='not_purchased')

---

In [230]:
item_purchases = pivoted.drop('user_no', axis=1).groupby('item_no').agg(np.sum)
user_interactions_with_item = pivoted['item_no'].value_counts()

item_summary = pd.merge(item_purchases,
                       user_interactions_with_item, 
                       left_index=True,
                       right_index=True).rename(columns={'item_no': 'user_interactions'})

item_summary['item_conversion'] = item_summary['purchased'] / item_summary['user_interactions']

In [233]:
item_summary = pd.merge(item_summary, item_info_df, how='left', left_index=True, right_on='item_no')

In [238]:
item_summary['colour'] = item_summary['colour'].apply(lambda x: x.strip())

In [239]:
item_summary.groupby('colour')['item_conversion'].describe()

,count,mean,std,min,25%,50%,75%,max
colour,,,,,,,,
aqua,2.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ash,1.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
beige,2694.0,0.258234,0.249747,0.000000,0.000000,0.225806,0.400000,2.000000
black,6359.0,0.248245,0.279397,0.000000,0.000000,0.192308,0.400000,2.000000
blue,9644.0,0.303916,0.302227,0.000000,0.000000,0.250000,0.500000,2.000000
brown,3011.0,0.261907,0.252258,0.000000,0.000000,0.239437,0.400000,1.000000
cream,2122.0,0.283625,0.278527,0.000000,0.000000,0.250000,0.428571,2.000000
dark grey,1.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
flame red,1.0,0.307692,NaN,0.307692,0.307692,0.307692,0.307692,0.307692


In [240]:
item_summary.groupby('gender_description')['item_conversion'].describe()

,count,mean,std,min,25%,50%,75%,max
gender_description,,,,,,,,
boys,11813.0,0.320463,0.315929,0.0,0.0,0.272727,0.5,2.0
girls,21253.0,0.291246,0.302700,0.0,0.0,0.250000,0.5,2.0
unisex,28637.0,0.258885,0.250718,0.0,0.0,0.230769,0.4,2.0


In [250]:
item_summary['brand'].value_counts().describe()

count     644.000000
mean       95.812112
std       199.671847
min         1.000000
25%         5.000000
50%        27.000000
75%        92.250000
max      2202.000000
Name: brand, dtype: float64

In [251]:
brand_summary = item_summary.groupby('brand')['item_conversion'].describe()
brand_summary = brand_summary.loc[brand_summary['count'] > 5]
brand_summary.sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
brand,,,,,,,,
boy london,7.0,0.785714,0.393398,0.0,0.75,1.000000,1.0,1.0
henri lloyd,7.0,0.714286,0.404995,0.0,0.50,1.000000,1.0,1.0
anïve for the minors,6.0,0.666667,0.516398,0.0,0.25,1.000000,1.0,1.0
g-star raw,18.0,0.611111,0.471405,0.0,0.00,1.000000,1.0,1.0
gcds,7.0,0.595238,0.449868,0.0,0.25,0.666667,1.0,1.0
...,...,...,...,...,...,...,...,...
gucci,11.0,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0
emmaljunga,43.0,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0
carriwell,16.0,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0


In [248]:
item_summary['product_group'].value_counts().describe()

count     104.000000
mean      593.298077
std      1178.452745
min         1.000000
25%        21.750000
50%       137.500000
75%       545.000000
max      7463.000000
Name: product_group, dtype: float64

In [254]:
product_group_summary = item_summary.groupby('product_group')['item_conversion'].describe()
product_group_summary = product_group_summary.loc[product_group_summary['count'] > 20]
product_group_summary.sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
product_group,,,,,,,,
hair accessories,575.0,0.439410,0.333335,0.0,0.045455,0.500000,0.666667,1.00
shorts,1145.0,0.401881,0.344386,0.0,0.000000,0.375000,0.600000,1.00
underwear,2204.0,0.387777,0.304713,0.0,0.142857,0.375000,0.571429,1.50
baby feeding,756.0,0.384390,0.271821,0.0,0.200000,0.379310,0.552682,1.25
musical instruments and toys,81.0,0.379388,0.305663,0.0,0.200000,0.333333,0.500000,1.00
...,...,...,...,...,...,...,...,...
changing and travel bags,268.0,0.146670,0.193391,0.0,0.000000,0.000000,0.250000,1.00
ride ons and walkers,60.0,0.139955,0.164393,0.0,0.000000,0.045804,0.268234,0.50
car seats,377.0,0.135372,0.166089,0.0,0.000000,0.097222,0.200000,1.00


---
## **KPIs**

 - [ ] KPIs 
     - [ ] Visitors per month
     - [ ] Conversion ratio
     - [ ] Total items sold (per month)
     - [ ] Sessions per month
     - [ ] Sessions with a purchase per month
     - [ ] Returned users 

**Unique Visitors per Month**

In [260]:
user_item_interaction_df['date'] = pd.to_datetime(user_item_interaction_df['date'])

In [267]:
interactions_by_month = user_item_interaction_df.groupby(user_item_interaction_df['date'].dt.month)


In [276]:
num_unique_visitors = [group[1]['user_no'].nunique() for group in interactions_by_month]

In [277]:
num_unique_visitors

[14824, 12504, 15940, 14393, 12692, 15617, 12397, 19834, 20373, 24456, 35760]

**Purchases Per Month**

In [280]:
num_purchases = [len(group[1].loc[group[1]['eventtype'] == 'purchased']) for group in interactions_by_month]

In [281]:
num_purchases

[15284, 11786, 17583, 16577, 14818, 19425, 13543, 22110, 20354, 25643, 50042]

**User-item interactions that include a purchase**

In [291]:
total_user_item_interactions = [group[1].groupby(['user_no', 'item_no']).ngroups for group in interactions_by_month]
user_item_interactions_w_purchase = []

In [292]:
total_user_item_interactions

[48404, 38009, 52334, 47711, 41352, 55091, 44392, 76123, 79652, 104497, 222892]

In [302]:
user_item_interactions_purchase = [group[1].loc[group[1]['eventtype'] == 'purchased'].groupby(['user_no', 'item_no']).ngroups
                                  for group in interactions_by_month]

In [303]:
user_item_interactions_purchase

[15157, 11685, 17367, 16376, 14604, 19172, 13428, 21840, 20078, 25145, 49699]

In [305]:
np.array(user_item_interactions_purchase) / np.array(total_user_item_interactions)

array([0.31313528, 0.30742719, 0.33184928, 0.34323322, 0.35316309,
       0.34800603, 0.30248693, 0.28690409, 0.25207151, 0.24062892,
       0.22297346])

---

## **Cold Start** 

Returned users/new users per month 
Repeat items/new items per month

## **Individual User Journey**

 - [ ] Dashboard
 - [ ] Buy same type of clothes?
 - [ ] Can we see them growing up?
 - [ ] Same brand, same gender?

## **Train Test Split** 

## **Baselines and Heuristics**

## **Features to Use**